In [1]:
def printMatrix(mat):
    for i in range(len(mat)):
        for j in range(len(mat[i])):
            print(mat[i][j], end=" ")
        print()

def constructMatrix(str1, str2):
    list = [[0 for i in range(len(str1)+1)] for j in range(len(str2)+1)]
    return list

def printStringMatrix(matrix, str1, str2):
    print("", end = "  ")
    print("_", end = " ")
    for i in range(len(str1)):
        print(str1[i], end = " ")
    print()
    for i in range(len(matrix)):
        if i == 0 :
            print("_", end = " ")
        else:
            print(str2[i-1], end = " ")
        for j in range(len(matrix[i])):
            print(matrix[i][j], end = " ")
        print()

def initMatrix(matrix, gapPenalty):
    x=0
    for i in range(len(matrix)):
        matrix[i][0] = x * gapPenalty
        x=x+1
    x=0
    for i in range(len(matrix[0])):
        matrix[0][i] = x * gapPenalty
        x=x+1
    return matrix

def getMinPenalty(matrix, i, j, str1, str2,matchPenalty, mismatchPenalty, gapPenalty ):
    if str2[i-1] == str1[j-1]:
        currPenalty = matchPenalty
    else:
        currPenalty = mismatchPenalty

    top = matrix[i-1][j] + gapPenalty
    left = matrix[i][j-1] + gapPenalty
    diag = matrix[i-1][j-1] + currPenalty
    maxE = max(top, left, diag)
    currPenalty = maxE

    return currPenalty

def fillMatrix(matrix, str1, str2, matchPenalty, mismatchPenalty, gapPenalty):
    for i in range(1, len(matrix)):
        for j in range(1, len(matrix[0])):
            matrix[i][j] = getMinPenalty(matrix, i, j, str1, str2, matchPenalty, mismatchPenalty, gapPenalty)


def getPrevious(matrix, i, j,matchPenalty, mismatchPenalty, gapPenalty ):

    score = matrix[i][j]
    top = matrix[i-1][j]
    left = matrix[i][j-1]
    diag = matrix[i-1][j-1]

    maxPrevIndexI = 0
    maxPrevIndexJ = 0
    maxPrevDir = 0
    if diag + matchPenalty == score or diag + mismatchPenalty == score:
        maxPrevIndexI = i-1
        maxPrevIndexJ = j-1
        maxPrevDir = 0
    elif top + gapPenalty == score:
        maxPrevIndexI = i-1
        maxPrevIndexJ = j
        maxPrevDir = 1
    elif left + gapPenalty == score:
        maxPrevIndexI = i
        maxPrevIndexJ = j-1
        maxPrevDir = 2

    return maxPrevDir, maxPrevIndexI, maxPrevIndexJ


def backTrack(matrix, matchPenalty, mismatchPenalty, gapPenalty):

    score = 0
    directions = []

    i = len(matrix)-1
    j = len(matrix[0])-1

    while i>=0 and j>=0:
        score += matrix[i][j]
        if i == 0 or j == 0:
            break
        prev = getPrevious(matrix, i, j, matchPenalty, mismatchPenalty, gapPenalty)
        directions.insert(0, prev[0])
        i = prev[1]
        j = prev[2]
    return directions, score


In [2]:
def alignSequences(sequence1, sequence2, matchPenalty = 1, mismatchPenalty = -1, gapPenalty = -2):
    mat = constructMatrix(sequence1, sequence2)
    matrix = initMatrix(mat, gapPenalty)
    fillMatrix(matrix, sequence1, sequence2, matchPenalty, mismatchPenalty, gapPenalty)
    dir = backTrack(matrix, matchPenalty, mismatchPenalty, gapPenalty)
    return dir[1]

In [6]:
alignSequences("GGGGGCCATCGTAATGTCACCTTT", "AAGGGCCATAGGTGTGTTAACCTC")

72

In [3]:
from Bio import SeqIO
sequences = {}
for record in SeqIO.parse("sequences_24.fasta", "fasta"):
    sequences[record.id] = str(record.seq)
print(len(sequences))

529


In [4]:
sequences

{'C001': 'GGGAGCCATCGTAATGTTTCCCCC',
 'C002': 'GGGGGCCACCGTAATGCCTCCCCC',
 'C003': 'GAGAGCCATAGGAGTGCCAACCTT',
 'C004': 'GAGGGCCATAGGAGAACTACTCTC',
 'C005': 'AGGGGCCATAGTAAAGCCAACTTC',
 'C006': 'GGGAGCGGTATTAATGTTACCTTT',
 'C007': 'GGGAGCGGTATTAATGTTACCTTT',
 'C008': 'GGGGGCGGTATGTATGTTACCTTC',
 'C009': 'GGGAGCCGTATGAATGTTACCTTT',
 'C010': 'GAGGGCCGTATGTATGCCAACCTT',
 'C011': 'GGGAGCGGTATGTATGTTACCTTT',
 'C012': 'GAAAGCCATAGGTGAGTTAACCTT',
 'C013': 'GGGGGCCACCGTAATGTCTCCCTC',
 'C014': 'GAAAGCGGTATGTGTACCAACTTT',
 'C015': 'GGGGGCCACAGTAATGTCTCCCCC',
 'C016': 'GAGAGCCATAGGTGTACCAACCCT',
 'C017': 'GAAGGCCATAGGTGTACCAACCTT',
 'C018': 'GAAAGCCATAGGTGTACCAACCTT',
 'C019': 'GGGAGCGGTATTAATGTTACCTTT',
 'C020': 'GGGAGCGGTAGTAATGTTACCTCT',
 'C021': 'GGGAGCGGTATTAATGCTACCTCT',
 'C022': 'GGGAGCGATCGGTATGTTACCTTT',
 'C023': 'GAGGGCCATAGGTGTACCAACCTT',
 'C024': 'GGAAGCGATCGTAATGTTAACTCT',
 'C025': 'GGGGGCGGTATGTATGTTACCTTT',
 'C026': 'GAAAGCCATAGGTGTACCAACCTT',
 'C027': 'GGGAGCGGTATGAATGTTACCTTC',
 

In [ ]:
# NNGGGNCANNNNNNNNNCNNNCTC
# GAGGGCCATAGGTATGTCACCTTT
# GAAAGCCATAGGNNTGCNAACNNT
# GGGGGCCACCGTAATGCCTCCCCC
# AA

In [172]:
querySequence = "AAAGCATA"
alignmentScore = dict()
for k,v in sequences.items():
    alignmentScore[k] = alignSequences(querySequence, v)
print(alignmentScore)

{'C001': -262, 'C002': -260, 'C003': -254, 'C004': -254, 'C005': -254, 'C006': -260, 'C007': -260, 'C008': -270, 'C009': -260, 'C010': -254, 'C011': -260, 'C012': -252, 'C013': -260, 'C014': -252, 'C015': -264, 'C016': -256, 'C017': -254, 'C018': -252, 'C019': -260, 'C020': -260, 'C021': -254, 'C022': -260, 'C023': -254, 'C024': -258, 'C025': -270, 'C026': -252, 'C027': -260, 'C028': -252, 'C029': -254, 'C030': -260, 'C031': -260, 'C032': -252, 'C033': -258, 'C034': -260, 'C035': -252, 'C036': -260, 'C037': -260, 'C038': -260, 'C039': -260, 'C040': -260, 'C041': -260, 'C042': -260, 'C043': -260, 'C044': -260, 'C045': -260, 'C046': -260, 'C047': -260, 'C048': -252, 'C049': -254, 'C050': -254, 'C051': -252, 'C052': -252, 'C053': -254, 'C054': -260, 'C055': -260, 'C056': -256, 'C057': -254, 'C058': -260, 'C059': -260, 'C060': -260, 'C061': -260, 'C062': -260, 'C063': -252, 'C064': -252, 'C065': -260, 'C066': -272, 'C067': -252, 'C068': -260, 'C069': -260, 'C070': -254, 'C071': -258, 'C072

In [173]:
sorted_dict_by_value_desc = dict(sorted(alignmentScore.items(), key=lambda item: item[1], reverse=True))
sorted_dict_by_value_desc

{'C012': -252,
 'C014': -252,
 'C018': -252,
 'C026': -252,
 'C028': -252,
 'C032': -252,
 'C035': -252,
 'C048': -252,
 'C051': -252,
 'C052': -252,
 'C063': -252,
 'C064': -252,
 'C067': -252,
 'C075': -252,
 'C082': -252,
 'C116': -252,
 'C119': -252,
 'C120': -252,
 'C138': -252,
 'C146': -252,
 'C149': -252,
 'C150': -252,
 'C152': -252,
 'C154': -252,
 'C172': -252,
 'C181': -252,
 'C185': -252,
 'W003': -252,
 'W010': -252,
 'W011': -252,
 'W012': -252,
 'W013': -252,
 'W016': -252,
 'W021': -252,
 'W027': -252,
 'W038': -252,
 'W052': -252,
 'W060': -252,
 'W061': -252,
 'W063': -252,
 'W088': -252,
 'W109': -252,
 'W117': -252,
 'W119': -252,
 'W120': -252,
 'W125': -252,
 'W128': -252,
 'W130': -252,
 'W158': -252,
 'W160': -252,
 'W162': -252,
 'W193': -252,
 'W207': -252,
 'W209': -252,
 'W226': -252,
 'W249': -252,
 'W250': -252,
 'W281': -252,
 'W283': -252,
 'W305': -252,
 'W323': -252,
 'W327': -252,
 'W328': -252,
 'C003': -254,
 'C004': -254,
 'C005': -254,
 'C010': -

In [175]:
k = 3

In [176]:
topk = list(sorted_dict_by_value_desc.keys())[:k]
topk

['C012', 'C014', 'C018']

In [177]:
import pandas as pd
df = pd.read_csv("mapping_24.csv")
df['Subpopulation']= df['Subpopulation'].astype("string")

In [178]:
print(df.columns)

Index(['Cultivar ID', 'vg0112116426', 'vg0128525986', 'vg0130976864',
       'vg0131664768', 'vg0133440209', 'vg0135617816', 'vg0135642980',
       'vg0138418739', 'vg0138428840', 'vg0138608956', 'vg0138999212',
       'vg0405463422', 'vg0405463763', 'vg0603483061', 'vg0713178880',
       'vg0719727299', 'vg0719727339', 'vg0719834473', 'vg0819793460',
       'vg0904094998', 'vg0904282939', 'vg1019044175', 'vg1123563633',
       'vg1207667840', 'Subpopulation', 'Plant Height (cm)', 'sequence'],
      dtype='object')


In [179]:
for cultivar in topk:
    row = df.loc[df['Cultivar ID'] == cultivar]
    print(cultivar, row['Subpopulation'].item() )

C012 Temperate Japonica
C014 Japonica Intermediate
C018 Temperate Japonica


In [180]:
import json

# Read the JSON data from a file
with open("json/clusters_dict.json", "r") as f:
    clusters_dict = json.loads(f.read())

with open("json/sequence_cluster_dict.json", "r") as f:
    sequence_cluster_dict = json.loads(f.read())

with open("json/sequences_dict.json", "r") as f:
    sequences_dict = json.loads(f.read())

with open("json/unique_sequences_per_cluster.json", "r") as f:
    unique_sequences_per_cluster = json.loads(f.read())


In [181]:
clusters = []
subpops = []
for cultivar in topk:
    row = df.loc[df['Cultivar ID'] == cultivar]
    print(cultivar, row['Subpopulation'].item(), sequence_cluster_dict[cultivar])
    clusters.append(sequence_cluster_dict[cultivar])
    subpops.append(row['Subpopulation'].item())

C012 Temperate Japonica 65
C014 Japonica Intermediate 1
C018 Temperate Japonica 20


In [182]:
cluster_set = set(clusters)
cluster_freq = dict()
for x in cluster_set:
    cluster_freq[x] = clusters.count(x)
cluster_freq = dict(sorted(cluster_freq.items(), key=lambda item: item[1], reverse=True))
cluster_belong = list(cluster_freq.keys())[0]
cluster_belong

65

In [183]:
subpop_set = set(subpops)
subpop_freq = dict()
for x in subpop_set:
    subpop_freq[x] = subpops.count(x)
subpop_freq = dict(sorted(subpop_freq.items(), key=lambda item: item[1], reverse=True))
subpop_belong = list(subpop_freq.keys())[0]
subpop_belong

'Temperate Japonica'

In [184]:
df_new = df[df['Cultivar ID'].isin(clusters_dict[str(cluster_belong)])]
df_new

,Cultivar ID,vg0112116426,vg0128525986,vg0130976864,vg0131664768,vg0133440209,vg0135617816,vg0135642980,vg0138418739,vg0138428840,...,vg0719834473,vg0819793460,vg0904094998,vg0904282939,vg1019044175,vg1123563633,vg1207667840,Subpopulation,Plant Height (cm),sequence
11,C012,G,A,A,A,G,C,C,A,T,...,T,A,A,C,C,T,T,Temperate Japonica,114.1,GAAAGCCATAGGTGAGTTAACCTT
257,W056,G,A,G,A,G,C,C,A,T,...,T,A,A,C,C,T,T,Temperate Japonica,129.0,GAGAGCCATAGGTGAGTTAACCTT


In [185]:
heights = df_new['Plant Height (cm)']
print("Height range : ", heights.min(), " - ", heights.max())


Height range :  114.1  -  129.0


In [74]:
def height_range(query_sequence, nk=5):
    alignmentScore = dict()
    for k,v in sequences.items():
        alignmentScore[k] = alignSequences(querySequence, v)
    sorted_dict_by_value_desc = dict(sorted(alignmentScore.items(), key=lambda item: item[1], reverse=True))
    topk = list(sorted_dict_by_value_desc.keys())[:nk]
    clusters = []
    subpops = []
    for cultivar in topk:
        row = df.loc[df['Cultivar ID'] == cultivar]
        clusters.append(sequence_cluster_dict[cultivar])
        subpops.append(row['Subpopulation'].item())
    cluster_set = set(clusters)
    cluster_freq = dict()
    for x in cluster_set:
        cluster_freq[x] = clusters.count(x)
    cluster_freq = dict(sorted(cluster_freq.items(), key=lambda item: item[1], reverse=True))
    cluster_belong = list(cluster_freq.keys())[0]
    subpop_set = set(subpops)
    subpop_freq = dict()
    for x in subpop_set:
        subpop_freq[x] = subpops.count(x)
    subpop_freq = dict(sorted(subpop_freq.items(), key=lambda item: item[1], reverse=True))
    subpop_belong = list(subpop_freq.keys())[0]
    df_new = df[df['Cultivar ID'].isin(clusters_dict[str(cluster_belong)])]
    heights = df_new['Plant Height (cm)']
    print("Height range : ", heights.min(), " - ", heights.max())

In [191]:
height_range("AAAGCATA", nk=3)

Height range :  114.1  -  129.0
